In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from haversine import haversine
import re
import gc
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb
import xgboost

import matplotlib.pyplot as plt
import seaborn as sns

## data load

In [2]:
train = pd.read_parquet('./train07.parquet') # 7월 only 
test = pd.read_parquet('./test.parquet')
print(train.shape, test.shape)

(274695, 23) (291241, 22)


## 데이터 전처리

### base_date
train_base_date_df = train.groupby(['base_date','road_name']).count()[['id']].reset_index()
test_base_date_df = test.groupby(['base_date','road_name']).count()[['id']].reset_index()

train_base_date_df.columns = ['base_date','road_name','base_date_cnt']
test_base_date_df.columns = ['base_date','road_name','base_date_cnt']

train = train.merge(train_base_date_df, how='inner', on=['base_date','road_name'])
test = test.merge(test_base_date_df, how='inner', on=['base_date','road_name'])

In [3]:
# 위경도로 거리 도출
%time
train["distance"] = train[["start_latitude", "start_longitude", "end_latitude", "end_longitude"]].apply(lambda x: haversine((x[0], x[1]), (x[2], x[3])), axis="columns")
test["distance"] = test[["start_latitude", "start_longitude", "end_latitude", "end_longitude"]].apply(lambda x: haversine((x[0], x[1]), (x[2], x[3])), axis="columns")

Wall time: 995 µs


In [4]:
# label encoding 
str_col = ['day_of_week', 'start_turn_restricted','end_turn_restricted'
          ]   
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [5]:
# one-hot
cols = ['road_type', 'start_turn_restricted', 'end_turn_restricted', 'lane_count', 'road_rating','weight_restricted', 'day_of_week'
       ]  

train = pd.get_dummies(train, prefix_sep='_', sparse=False, drop_first=True, columns = cols)
test = pd.get_dummies(test, prefix_sep='_', sparse=False, drop_first=True, columns = cols)

In [6]:
# del 
del_cols = ['id', 'connect_code','height_restricted','vehicle_restricted'
            , 'start_node_name', 'end_node_name', 'road_name'
           ]  # , 'start_node_name', 
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

In [ ]:
train.corr()[['target']].abs().sort_values('target', ascending=False)

In [ ]:
#train[(train['base_date'] == 20220701) & (train['base_hour'] == 0)].sort_values('target').to_excel('temp/20220701_00.xlsx', encoding='utf-8-sig') 

## 모델링

In [7]:
gc.collect()

40

In [ ]:
print(train.columns)
print(test.columns)

In [8]:
from pycaret.regression import *

In [ ]:
sup = setup(train, target = 'target', session_id=42)

In [ ]:
best_3 = compare_models(sort = 'MAE', n_select=3, fold = 3, exclude=['knn','ada','lar','lr','ridge','br','omp','lasso','en','huber','llar','par'])

In [ ]:
best_3

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
X_train = pd.concat([train, test])

In [24]:
train

,base_date,base_hour,multi_linked,maximum_speed_limit,start_latitude,start_longitude,end_latitude,end_longitude,target,distance,road_type_3,start_turn_restricted_1,end_turn_restricted_1,lane_count_2,lane_count_3,road_rating_106,road_rating_107,weight_restricted_32400.0,weight_restricted_43200.0,weight_restricted_50000.0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
1,20220728,21,0,60.0,33.500730,126.529107,33.504811,126.526240,30.0,0.525891,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
11,20220724,2,0,50.0,33.248505,126.569797,33.248633,126.567766,40.0,0.189391,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
16,20220701,22,0,50.0,33.485885,126.489979,33.485975,126.486409,35.0,0.331239,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
20,20220701,21,0,70.0,33.500103,126.512851,33.500132,126.512046,21.0,0.074720,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,20220702,19,0,50.0,33.485704,126.496451,33.483589,126.496368,24.0,0.235388,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701149,20220711,19,0,50.0,33.251092,126.435439,33.251095,126.435138,41.0,0.028025,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
4701153,20220720,23,0,60.0,33.251426,126.509066,33.251045,126.510574,28.0,0.146448,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0
4701164,20220708,9,0,70.0,33.471061,126.545467,33.469352,126.547314,27.0,0.255851,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4701178,20220731,15,0,60.0,33.290116,126.489006,33.289861,126.487409,52.0,0.151111,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0


In [13]:
from datetime import datetime, timedelta

In [16]:
from tqdm import tqdm

In [28]:
start_date = pd.to_datetime('20220630')
end_date = pd.to_datetime('20220730')
pred_date = pd.to_datetime('20220731')

for d in tqdm(range(1, 32)):

    p_start_date = int((start_date + timedelta(days=d)).strftime('%Y%m%d'))
    p_end_date = int((end_date + timedelta(days=d)).strftime('%Y%m%d'))
    p_pred_date = int((pred_date + timedelta(days=d)).strftime('%Y%m%d'))
    print(d, p_start_date, p_end_date, p_pred_date)
    
    model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)
    
    train_temp = train[(train['base_date'] >= p_start_date) & (train['base_date'] <= p_end_date)]
    model.fit(train_temp.drop('target', axis=1), train_temp['target'])
    
    test_temp = test[test['base_date'] == p_pred_date]
    pred = pd.DataFrame(model.predict(test_temp))
    
    pred = pd.concat([test_temp.reset_index(), pred], axis=1, ignore_index=True)
    
    pred.columns = ['index', 'base_date', 'base_hour', 'multi_linked', 'maximum_speed_limit',
       'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude',
       'distance', 'road_type_3', 'start_turn_restricted_1',
       'end_turn_restricted_1', 'lane_count_2', 'lane_count_3',
       'road_rating_106', 'road_rating_107', 'weight_restricted_32400.0',
       'weight_restricted_43200.0', 'weight_restricted_50000.0',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'day_of_week_5', 'day_of_week_6', 'target']
    
    pred = pred.set_index('index')
    
    train = pd.concat([train_temp, pred]) 

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

1 20220701 20220731 20220801


  3%|██▋                                                                                | 1/31 [00:49<24:56, 49.89s/it]

2 20220702 20220801 20220802


  6%|█████▎                                                                             | 2/31 [01:40<24:13, 50.12s/it]

...

 94%|████████████████████████████████████████████████████████████████████████████▋     | 29/31 [24:29<01:42, 51.23s/it]

30 20220730 20220829 20220830


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 30/31 [25:22<00:51, 51.79s/it]

31 20220731 20220830 20220831


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [26:16<00:00, 50.87s/it]


In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5)

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)

In [29]:
sample_submission = pd.read_csv('input/sample_submission.csv')

In [41]:
sample_submission = pd.concat([sample_submission[['id']], train[train.base_date >= 20220801][['target']]], axis=1 )

In [42]:
sample_submission.to_csv("output/20221030-1.csv", index = False)